In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data, 'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
column_names = ['PostalCode', 'Borough', 'Neighborhood']
rows_list = []
for tr in My_table.findAll('tr'):
    tds = tr.findAll('td')
    if not tds:
        continue
    postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
    rows_list.append([postcode, borough, neighborhood])
postalcodes = pd.DataFrame(rows_list, columns=column_names)

indexNames = postalcodes[postalcodes['Borough'] == 'Not assigned'].index
postalcodes.drop(indexNames, inplace=True)
postalcodes = postalcodes.reset_index()
del postalcodes['index']
postalcodes.loc[postalcodes['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = postalcodes['Borough']
postalcodes_group = postalcodes.groupby(['PostalCode', 'Borough'], sort=False).agg(lambda x: ', '.join(x))
postalcodes_group = postalcodes_group.reset_index()

In [5]:
number_of_rows = postalcodes_group.shape[0]
number_of_rows

103

In [7]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [8]:
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
merged_dataframe = pd.merge(postalcodes_group, coordinates_df, how='left', on='PostalCode')
merged_dataframe.head()
#merged_dataframe.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [14]:
# select only toronto
merged_dataframe_Toronto = merged_dataframe[merged_dataframe['Borough'].str.contains("Toronto")]
merged_dataframe_Toronto = merged_dataframe_Toronto.reset_index()
del merged_dataframe_Toronto['index']
merged_dataframe_Toronto.head()
#merged_dataframe_Toronto.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
